In [9]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [10]:
#loading in the csv data
# df = pd.read_csv('c:/data/styles.csv')
df = pd.read_csv('c:/data/styles.csv',on_bad_lines='skip')
df.head()

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt


In [11]:
df = df.dropna()
df.nunique()
df.columns

Index(['id', 'gender', 'masterCategory', 'subCategory', 'articleType',
       'baseColour', 'season', 'year', 'usage', 'productDisplayName'],
      dtype='object')

In [12]:
# Looking at all the unique labels in all categorical columns 
cat_columns = ['gender', 'masterCategory', 'subCategory', 'articleType','baseColour', 'season', 'year', 'usage']

for col in cat_columns:
    print(col)
    print(df[col].unique())
    print('-------------------------')

gender
['Men' 'Women' 'Boys' 'Girls' 'Unisex']
-------------------------
masterCategory
['Apparel' 'Accessories' 'Footwear' 'Personal Care' 'Free Items'
 'Sporting Goods' 'Home']
-------------------------
subCategory
['Topwear' 'Bottomwear' 'Watches' 'Socks' 'Shoes' 'Belts' 'Flip Flops'
 'Bags' 'Innerwear' 'Sandal' 'Shoe Accessories' 'Fragrance' 'Jewellery'
 'Lips' 'Saree' 'Eyewear' 'Scarves' 'Dress' 'Loungewear and Nightwear'
 'Wallets' 'Apparel Set' 'Headwear' 'Mufflers' 'Skin Care' 'Makeup'
 'Free Gifts' 'Ties' 'Accessories' 'Nails' 'Beauty Accessories'
 'Water Bottle' 'Skin' 'Eyes' 'Bath and Body' 'Gloves'
 'Sports Accessories' 'Cufflinks' 'Sports Equipment' 'Stoles' 'Hair'
 'Perfumes' 'Home Furnishing' 'Umbrellas' 'Wristbands' 'Vouchers']
-------------------------
articleType
['Shirts' 'Jeans' 'Watches' 'Track Pants' 'Tshirts' 'Socks' 'Casual Shoes'
 'Belts' 'Flip Flops' 'Handbags' 'Tops' 'Bra' 'Sandals' 'Shoe Accessories'
 'Sweatshirts' 'Deodorant' 'Formal Shoes' 'Bracelet' 'Lips

The images in this Dataset are very low resolution (80x60). We will be using the categories that are visually distinct even at such a low resolution.

The categories year, usage, season, and gender mighht not be clearly visually distinct in some cases, so we wont be using them.

The categories, masterCategory and subCategory are distinct enough groups, but they are not specific enough for practical use. 

The categories we will use are articleType and baseColour(You can use more categories if you want).

There are many unique labels in these categories, we will only be using the ones with more than 1000 examples, since we would need a good number of samples for proper classifcation.

In [13]:
value_counts = df['articleType'].value_counts()

indexes = value_counts.index

values = value_counts.values

for i in range(len(value_counts)):

    if values[i] <1000:
        break

types_used = indexes[:i]
print('Article types used: ',types_used)

Article types used:  Index(['Tshirts', 'Shirts', 'Casual Shoes', 'Watches', 'Sports Shoes',
       'Kurtas', 'Tops', 'Handbags', 'Heels', 'Sunglasses'],
      dtype='object', name='articleType')


In [14]:
value_counts = df['baseColour'].value_counts()

indexes = value_counts.index

values = value_counts.values

for i in range(len(value_counts)):

    if values[i] <1000:
        break

colours_used = indexes[:i]
print('Base Colours used: ',colours_used)

Base Colours used:  Index(['Black', 'White', 'Blue', 'Brown', 'Grey', 'Red', 'Green', 'Pink',
       'Navy Blue', 'Purple', 'Silver'],
      dtype='object', name='baseColour')


In [15]:
# Removing all the examples with labels other than the selected ones
 
df = df[df['articleType'].isin(types_used)]
df = df[df['baseColour'].isin(colours_used)]

In [16]:
#number of examples we are left with
len(df)

21835

Now we will load in all the images from the remaining rows, and convert them to numpy arrays with img_to_array function in keras.

In [35]:
# %pip install opencv-python

     --------------------------------------- 38.1/38.1 MB 46.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [18]:
data = []

# Reading all the images and processing the data in them 

from tensorflow.keras.preprocessing.image import img_to_array
import cv2

IX = 80
IY = 60

invalid_ids = []

for name in df.id:

    try:
        image = cv2.imread('/kaggle/input/fashion-product-images-small/myntradataset/images/'+str(name)+'.jpg')
        image = cv2.resize(image, (IX,IY) )
        image = img_to_array(image)
        data.append(image)        
    except: 
        # Images for certain ids are missing, so they are not added to the dataset  
        invalid_ids.append(name)

In [19]:
# ids of missing images
print('invalid ids:')
print(invalid_ids)

invalid ids:
[15970, 59263, 53759, 1855, 30805, 26960, 30039, 9204, 47957, 12369, 29928, 42419, 47359, 17429, 7990, 21977, 37812, 4729, 56825, 39988, 20099, 58183, 3954, 3168, 9452, 44984, 45856, 5891, 38630, 4943, 11188, 10866, 15528, 45258, 22198, 29570, 6628, 26538, 35913, 9660, 2288, 18237, 54118, 2872, 49653, 16957, 13419, 44970, 33822, 5865, 29584, 34009, 19311, 42089, 22950, 49461, 9694, 23882, 34835, 11518, 59297, 31923, 54588, 8322, 13841, 12993, 7158, 8110, 7964, 58177, 16392, 17240, 6617, 3365, 42042, 23849, 43190, 54127, 24250, 32335, 56670, 5092, 10859, 15517, 4140, 36795, 24062, 45869, 21510, 51667, 22395, 6425, 31782, 42270, 12500, 30650, 21948, 40143, 24406, 4716, 14293, 47366, 6041, 23623, 17416, 9835, 12958, 9009, 3533, 26351, 54543, 3701, 29917, 32751, 10401, 4524, 37223, 53766, 25520, 32597, 57100, 36137, 39943, 50325, 8913, 16530, 22705, 7167, 27277, 12190, 19578, 27879, 7355, 43500, 13270, 31120, 15187, 14055, 20260, 28469, 34036, 8779, 3391, 43164, 2243, 11143, 4

In [20]:
labels = []

used_columns = ['subCategory','baseColour']

# getting labels for the columns used

for index, row in df.iterrows():

    if row['id'] in invalid_ids:
        continue

    tags = []

    for col in used_columns:
        tags.append(row[col])

    labels.append(tags)

In [21]:
import numpy as np

# converting data into numpy arrays

data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

print(labels)

[]


Now we will create binary vectors as the outputs of the model

In [22]:
from sklearn.preprocessing import MultiLabelBinarizer

# creating a binary vector for the input labels 

mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(labels)

print(mlb.classes_)
print(labels[0])

[]


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D

inputShape = (IY, IX, 3)

# A very simple sequential model is used since the images are very low resolution and the categories are fiarly distinct

model = Sequential()

model.add(Conv2D(32, (3, 3), padding="same",input_shape=inputShape))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Flatten()) 

model.add(Dense(128))
model.add(Activation('sigmoid'))


out = len(mlb.classes_)

model.add(Dense(out))
model.add(Activation('sigmoid')) # activation function for the final layer has to be sigmoid, since mutiple output labels can have value 1
model.compile(loss='binary_crossentropy', # loss function has to be binary_crossentropy, it is calculated seperately for each of the outputs
              optimizer='adam',
              metrics=['mse'])

In [ ]:
from sklearn.model_selection import train_test_split

# splitting data into testing and training set 

(trainX, testX, trainY, testY) = train_test_split(data,labels, test_size=0.1, random_state=42)

In [ ]:
batch = 32
E = 50

#training the model 
model.fit(x=trainX,y=trainY,
          epochs=E ,verbose=1)

In [ ]:
preds = model.predict(testX)


# since the predictions of the model are sigmoid, we will first binarize them to 0 or 1
pred_binarized = []

for pred in preds:
    vals = []
    for val in pred:
        if val > 0.5:
            vals.append(1)
        else:
            vals.append(0)
    pred_binarized.append(vals) 

pred_binarized = np.array(pred_binarized)   


# we convert the output vectors to the predicted labels
true_test_labels = mlb.inverse_transform(testY)
pred_test_labels = mlb.inverse_transform(pred_binarized)

correct = 0
wrong = 0

# Evaluating the predictions of the model

for i in range(len(testY)):

    true_labels = list(true_test_labels[i])

    pred_labels = list(pred_test_labels[i])

    label1 = true_labels[0]
    label2 = true_labels[1]

    if label1 in pred_labels:
        correct+=1
    else:
        wrong+=1

    if label2 in pred_labels:
        correct+=1
    else:
        wrong+=1    



print('correct: ', correct)
print('missing/wrong: ', wrong)
print('Accuracy: ',correct/(correct+wrong))

In [ ]:
for i in range(20):
    print('True labels: ',true_test_labels[i],' Predicted labels: ',pred_test_labels[i])